# __Task Overview__

You're a marketing analyst and you've been told by the Chief Marketing Officer that recent marketing campaigns have not been as effective as they were expected to be. You need to analyze the data set to understand this problem and propose data-driven solutions.

# __Section 01: Exploratory Data Analysis__

Are there any null values or outliers? How will you wrangle/handle them?
Are there any variables that warrant transformations?
Are there any useful variables that you can engineer with the given data?
Do you notice any patterns or anomalies in the data? Can you plot them?

# __Section 02: Statistical Analysis__

Please run statistical tests in the form of regressions to answer these questions & propose data-driven action recommendations. Make sure to interpret your results with non-statistical jargons.

- What factors are significantly related to the number of store purchases?
- Does US fare significantly better than the Rest of the World in terms of total purchases?
- Your supervisor insists that people who buy gold are more conservative. Therefore, people who spent an above average amount on gold in the last 2 years would have more in store purchases. Justify or refute this statement using an appropriate statistical test
- Fish has Omega 3 fatty acids which are good for the brain. Accordingly, do "Married PhD candidates" have a significant relation with amount spent on fish? What other factors are significantly related to amount spent on fish? (Hint: use your knowledge of interaction variables/effects)
- Is there a significant relationship between geographical regional and success of a campaign?

# __Section 03: Data Visualization__

Please plot and visualize the answers to the below questions.
- Which marketing campaign is most successful?
- What does the average customer look like for this company?
- Which products are performing best?
- Which channels are underperforming?

# __Section 04: Recommendations__

Bring together everything from Sections 01 to 03 and provide data-driven recommendations/suggestions.

__Columns Explanation__

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F1327132%2F0c83aef015cef72a1c6ef0bdb18937f3%2Fcolumns%20meaning%20small.png?generation=1609671531255584&alt=media)

In [ ]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import gc

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, roc_auc_score, f1_score, mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.linear_model import LogisticRegression, LinearRegression

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

plt.rcParams["figure.figsize"] = (12,8)
plt.rcParams['axes.titlesize'] = 16
plt.style.use('seaborn-whitegrid')
sns.set_palette("Set3")


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from time import time, strftime, gmtime
start = time()
import datetime
print(str(datetime.datetime.now()))

In [ ]:
base_dir = '/kaggle/input/marketing-data/'

In [ ]:
df = pd.read_csv(base_dir + 'marketing_data.csv')
df.head()

In [ ]:
print(f'No. of rows in the dataset: {df.shape[0]}\nNo. of columns in the dataset: {df.shape[1]}')

In [ ]:
df.describe().T

In [ ]:
df.info()

__Initial Observations__
- There are Null values in feature 'Income'
- There are 5 categorical features (object) nad 23 numerical features
- Income is under object datatype - need to change it into numerical feature

# __EDA__

__1.1 Null Value and Outlier Analysis__

In [ ]:
missing = df.isna().sum().reset_index()
missing.columns = ['features', 'total_missing']
missing['percent'] = (missing['total_missing'] / len(df)) * 100
missing.index = missing['features']
del missing['features']

missing['total_missing'].plot(kind = 'bar')
plt.title('Missing Values Count')
missing.T

 - Null values in 'Income' need to be imputed
 - Before that we need to fix the spaces in its name and change the datatype from object to float

In [ ]:
df.rename(columns = {' Income ': 'Income'}, inplace = True)
df['Income'] = df['Income'].str.replace('$', '')
df['Income'] = df['Income'].str.replace(',', '').astype('float')

In [ ]:
plt.figure(figsize = (10, 6))
sns.histplot(data = df['Income'])
plt.title('Income Distribution')
plt.grid()

- The Income is distributed between 0 and 100,000 with few outliers, let's check what those outliers are

In [ ]:
income_outliers = df['Income'][df['Income'] > df['Income'].mean() + 3 * df['Income'].std()]
income_outliers

- There are 8 outlier values as shown above
- If we impute the NaNs with mean value it'll get affected by these outliers, so its better we impute using median values

In [ ]:
df['Income'] = df['Income'].fillna(df['Income'].median())

print(f'Number of Null values in *Income* after Imputation: {df["Income"].isna().sum()}')

__Check outliers for other features__

In [ ]:
cols_to_check = [c for c in df.columns if (df[c].dtype != 'object') & (df[c].nunique() > 2) & (c != 'ID')]
#cols_to_check.__len__()

fig, ax = plt.subplots(4, 4, figsize = (16, 10))
ax = ax.flatten()

for i, c in enumerate(cols_to_check):
    sns.boxplot(x = df[c], ax = ax[i])
plt.suptitle('Outlier Analysis using BoxPlots', fontsize = 25)
fig.tight_layout()

del cols_to_check
gc.collect()

- From the box plot of Year_Birth, there are 1900 and less than 1900 values which can be removed or imputed with other values
- Other outliers could be considered as inherent to the data (real world data)

In [ ]:
df[df['Year_Birth'] <= 1900]

- There are 3 rows with the outliers, we can remove these or impute them with most frequent occurances

In [ ]:
#Remove
df = df[df['Year_Birth'] > 1900].reset_index(drop = True)
#Impute
#out = df['Year_Birth'][df['Year_Birth'] <= 1900].index
#df['Year_Birth'][out] = np.nan
#df['Year_Birth'].fillna(df['Year_Birth'].value_counts().index[0], inplace = True)
#df.iloc[out]
df.head()

__Unique Value Count Plots__

- Columns 'ACCEPTE*' seems to be categorical feature with 2 values, let's plot their unique value counts

In [ ]:
countplot = ['Education', 'Marital_Status', 'Kidhome', 'Teenhome', 'Complain', 'Response', 'Country', 
             'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
fig, ax = plt.subplots(int(len(countplot) / 2), 2, figsize = (15, 20))
ax = ax.flatten()
for i, c in enumerate(countplot):
    sns.countplot(x = c, data = df, ax = ax[i])
plt.suptitle('Unique Value Count Plot', y = 1.0, fontsize = 25)
fig.tight_layout()


__1.2 Data Transformation__

__Dt_Customer__
- Dt_Customer column has datatype as object which can be changed to pandas datatime datatype.
- We can extract more features by changing to datatime type

In [ ]:
print(f'Before Transformation:\n{df["Dt_Customer"].head()}')

In [ ]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])
print(f'After Transformation:\n{df["Dt_Customer"].head()}')

__Education__

Let's create a new feature 'Customer_Age' and make some plots to explore more

In [ ]:
df['Customer_Age'] = df['Dt_Customer'].dt.year - df['Year_Birth']

sns.histplot(data = df['Customer_Age'], kde = True)
plt.title('Histogram Distribution of Customer Age');

In [ ]:
plt.figure(figsize = (20, 15))
sns.boxplot(data = df, x = 'Country', y = 'Customer_Age', hue = 'Education')
plt.ylim(5, 80)
plt.title('Country Vs Customer Age Per Education');

In [ ]:
sns.boxplot(data = df, x = 'Education', y = 'Income')
plt.ylim(0, 300000)
plt.title('Education Vs Income');

- From the Boxplot Education Vs Income, we can assume that 'Basic' is someone who Primary/Secondary school educated
- From the Country Vs Customer Age Per Education plot we can assume 'Graduation' as someone who hasn't completed thier undergrad or who is in the process of completing it
- Also '2n Cycle' could be construted as undergraduates from their income band and age. 

In [ ]:
df['Education'] = df['Education'].apply(lambda x: 'Undergraduate' if str(x) == '2n Cycle' else str(x))
sns.countplot(data = df, x = 'Education')
plt.title('Number of unique values plot after transformation - Education');

__Marital Status__
- In the Marital_Status feature we can assume 'YOLO', 'Alone' and 'Absurd' as 'Single'

In [ ]:
df['Marital_Status'] = df['Marital_Status'].apply(lambda x: 'Single' if str(x) in ['YOLO', 'Alone', 'Absurd'] else str(x))
sns.countplot(data = df, x = 'Marital_Status')
plt.title('Number of unique values plot after transformation - Marital_Status');

__1.3 Feature Engineering__

- On the outset we can create day, month and year feature from Dt_Customer
- Create number of children/dependents in home by adding 'Kidhome' and 'Teenhome' features
- Create number of Total_Purchases by adding all the purchases features
- Create TotalAmount_Spent by adding all the Mnt* features

In [ ]:
df['Dt_Customer_Month'] = df['Dt_Customer'].dt.month
df['Dt_Customer_Year'] = df['Dt_Customer'].dt.year

df['Num_Dependants'] = df['Kidhome'] + df['Teenhome']

purchase_features = [c for c in df.columns if 'Purchase' in str(c)]
#we should remove 'NumDealsPurchases' from the list above
purchase_features.remove('NumDealsPurchases')
df['Num_TotalPurchases'] = df[purchase_features].sum(axis = 1)

amt_spent_features = [c for c in df.columns if 'Mnt' in str(c)]
df['TotalAmount_Spent'] = df[amt_spent_features].sum(axis = 1)

#df.head()

In [ ]:
print(f'Avg. number of total purchases: {df["Num_TotalPurchases"].mean()}')
sns.histplot(data = df, x = 'Num_TotalPurchases', kde = True);

- We can create a categorical feature using the customer's age by binnning them, this could be helpful to understand purchaing behaviour
- Ref: https://www.kasasa.com/articles/generations/gen-x-gen-y-gen-z

In [ ]:
print(f'Min. Customer Age: {df["Customer_Age"].min()}')
print(f'Max. Customer Age: {df["Customer_Age"].max()}')

In [ ]:
df['AgeGroup'] = pd.cut(df['Customer_Age'], bins = [6, 24, 29, 40, 56, 75], 
                        labels = ['Gen-Z', 'Gen-Y.1', 'Gen-Y.2', 'Gen-X', 'BBoomers'])

sns.countplot(data = df, x = 'AgeGroup');
plt.title('Unique Count Plot by AgeGroup')

- The 'Accepted*', 'Response' and 'Complain' features are already One-Hot encoded
- We need to encode other categorical features

In [ ]:
df_orig = df.copy()

In [ ]:
encode_features = ['Education', 'Marital_Status', 'Country', 'AgeGroup']
print(f'Features that needs to be Label Encoded: \n{encode_features}')

for c in encode_features:
    lbl = LabelEncoder()
    lbl.fit(list(df[c].astype(str).values))
    df[c] = lbl.transform(list(df[c].astype(str).values))
print('Label Encoding done..')

categorical_features = ['AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 
                        'Response', 'Complain']
categorical_features.extend(encode_features)
print(f'Number of Categorical features: {len(categorical_features)}')

In [ ]:
numerical_features = [c for c in df.columns if c not in categorical_features]

print(f'Number of Numerical features: {len(numerical_features)}')
print(f'Number of Categorical features: {len(categorical_features)}')

# __Statistical Analysis__

__2.1 What factors are significantly related to the number of store purchases?__

- Let's check correlation of Total Purchases with other features

In [ ]:
df.corr()[['Num_TotalPurchases']].style.background_gradient(sns.light_palette('#2ecc71', as_cmap = True))

 - Income and Total amount spent are the most important factor that decides the total purchases feature (positive correlation)
 - Also NumWebPurchases, NumCatalogPurchases, NumStorePurchases have positive effect on the Total purhcases
 - NumWebVisitsMonth, Year_Birth, Kidhome have negative effect on Total Purchases

In [ ]:
sns.lineplot(data = df, y = 'Income', x = 'Num_TotalPurchases');

In [ ]:
sns.boxplot(data = df, y = 'Num_TotalPurchases', x = 'Num_Dependants');

- As the number of Dependents increase in a household, the Total purchases is lesser than others

- Let's have a regression model predict the Total number of purchases and from that we can find out the important features that have positive/negative effect on them

In [ ]:
features = df.drop(['ID', 'Num_TotalPurchases', 'Dt_Customer', 'Year_Birth'], axis = 1)
target = df['Num_TotalPurchases']

Xtrain, Xvalid, ytrain, yvalid = train_test_split(features, target, 
                                                  test_size = 0.2, random_state = 42)
print(Xtrain.shape, ytrain.shape, Xvalid.shape, yvalid.shape)

In [ ]:
print(f'Avg. Num_TotalPurchases: {df["Num_TotalPurchases"].mean()}')
print(f'Median Num_TotalPurchases: {df["Num_TotalPurchases"].median()}')

In [ ]:
lin_reg = LinearRegression()

lin_reg.fit(Xtrain, ytrain)
predictions = lin_reg.predict(Xvalid)
print(f'RMSE using Linear reg: {np.sqrt(mean_squared_error(yvalid, predictions))}')

sns.histplot(predictions, kde = True);

In [ ]:
print(f'Avg. Predictions: {np.mean(predictions)}')
print(f'Median Predictions: {np.median(predictions)}')

__ELI5 Interpretation__

- A feature is “important” if shuffling its values increases the model error, because in this case the model relied on the feature for the prediction. A feature is “unimportant” if shuffling its values leaves the model error unchanged, because in this case the model ignored the feature for the prediction.
- The first number in each row shows how much model performance decreased with a random shuffling.
- The number after the ± measures how performance varied from one-reshuffling to the next

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(lin_reg, random_state = 42).fit(Xvalid, yvalid)
eli5.show_weights(perm, feature_names = Xvalid.columns.tolist(), top = 10)

__SHAP Interpretation__

Here, all the values on the left represent the observations that shift the predicted value in the negative direction while the points on the right contribute to shifting the prediction in a positive direction.

In [ ]:
import shap

explainer = shap.Explainer(lin_reg, Xtrain)
shap_values = explainer(Xvalid)

#plotting
shap.summary_plot(shap_values, Xvalid)

- Store/Catlog/Web Purchases have positive correlation with number of purchases

- Now let's look at the features influencing the NumStorePurchases.

In [ ]:
sns.histplot(data = df['NumStorePurchases'], kde = True, palette = 'Set3');

In [ ]:
print(f'Avg. NumStorePurchases: {df["NumStorePurchases"].mean()}')
print(f'Median NumStorePurchases: {df["NumStorePurchases"].median()}')

In [ ]:
df.drop('ID', axis = 1).corr()[['NumStorePurchases']].style.background_gradient(sns.light_palette('#2ecc71', as_cmap = True))

In [ ]:
features = df.drop(['ID', 'NumStorePurchases', 'Dt_Customer', 'Year_Birth'], axis = 1)
target = df['NumStorePurchases']

Xtrain, Xvalid, ytrain, yvalid = train_test_split(features, target, 
                                                  test_size = 0.2, random_state = 42)
print(Xtrain.shape, ytrain.shape, Xvalid.shape, yvalid.shape)

In [ ]:
lin_reg = LinearRegression()

lin_reg.fit(Xtrain, ytrain)
predictions = lin_reg.predict(Xvalid)
print(f'RMSE using Linear reg: {np.sqrt(mean_squared_error(yvalid, predictions))}')

sns.histplot(predictions, kde = True);

In [ ]:
print(f'Avg. NumStorePurchases Prediction: {np.mean(predictions)}')
print(f'Median NumStorePurchases Prediction: {np.median(predictions)}')

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(lin_reg, random_state = 42).fit(Xvalid, yvalid)
eli5.show_weights(perm, feature_names = Xvalid.columns.tolist(), top = 10)

- Using Linear Regression gives us closer to the trainset predictions (avg. and median scores)
- Most Influencial features are Num_TotalPurchases, Num_WebPurchases, Num_CatalogPurchases

In [ ]:
import shap

explainer = shap.Explainer(lin_reg, Xtrain)
shap_values = explainer(Xvalid)

#plotting
shap.summary_plot(shap_values, Xvalid)

- When Catalog and Web purchases increase there is in reduction in number of store purchases

__2.2 Does US fare significantly better than the Rest of the World in terms of total purchases?__

In [ ]:
df_cn = pd.pivot_table(data = df_orig, index = 'Country', values = ['Num_TotalPurchases'], 
                        aggfunc = ['sum', 'mean'])
df_cn.T

In [ ]:
df_cn['sum']['Num_TotalPurchases'].sort_values(ascending = False).plot(kind = 'bar', rot = 0)
plt.ylabel('Total Num Purchases')
plt.title('Total Number of Purchases by Country');

- As can be seen from above pivot table, Spain (SP) has the most number of purchases and US in at the 7th place
- We can gain more insights into the purchase pattern among AgeGroup, Marital_Status and Education of the customers

In [ ]:
df_cn['mean']['Num_TotalPurchases'].plot(kind = 'bar', rot = 0)

__2.3 Find relationship between Amount spent in gold products and Store Purchases__

In [ ]:
sns.lineplot(data = df, x = 'NumStorePurchases', y = 'MntGoldProds', palette = 'Set3')

In [ ]:
df[['MntGoldProds', 'NumStorePurchases']].corr()

- There is a positive correlation between the number of store purchases and the amount spent on gold
- Usually people like to buy gold in store so that they can feel it and try it on them

__2.4 Fish has Omega 3 fatty acids which are good for the brain. Accordingly, do "Married PhD candidates" have a significant relation with amount spent on fish? What other factors are significantly related to amount spent on fish?__

In [ ]:
print(f'Avg. MntFishProducts: {np.mean(df["MntFishProducts"])}')
print(f'Median MntFishProducts: {np.median(df["MntFishProducts"])}')

In [ ]:
sns.histplot(data = df['MntFishProducts'], kde = True, palette = 'Set3')

In [ ]:
df_fish = pd.pivot_table(data = df_orig, index = ['Marital_Status', 'Education'], values = ['MntFishProducts'], 
                        aggfunc = 'sum')
df_fish.T

In [ ]:
df_fish.sort_values(by = 'MntFishProducts', ascending = False).plot(kind = 'bar', rot = 60)

- As can be seen from the pivot table chart, Married-Graduation category spends more on Fish products
- Married-PhD category spending on fish products is at 4th place
- Let's find what others factors influence spending on fish products

In [ ]:
df_orig.corr()[['MntFishProducts']].style.background_gradient(sns.light_palette('#2ecc71', as_cmap = True))

In [ ]:
features = df.drop(['ID', 'MntFishProducts', 'Dt_Customer', 'Year_Birth'], axis = 1)
target = df['MntFishProducts'].copy()

Xtrain, Xvalid, ytrain, yvalid = train_test_split(features, target, 
                                                  test_size = 0.2, random_state = 42)
print(Xtrain.shape, ytrain.shape, Xvalid.shape, yvalid.shape)

lin_reg = LinearRegression()

lin_reg.fit(Xtrain, ytrain)
predictions = lin_reg.predict(Xvalid)
print(f'RMSE using Linear reg: {np.sqrt(mean_squared_error(yvalid, predictions))}')

In [ ]:
print(f'Avg. MntFishProducts Predictions: {np.mean(predictions)}')
print(f'Median MntFishProducts Predictions: {np.median(predictions)}')

sns.histplot(predictions, kde = True).set(ylabel = None)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(lin_reg, random_state = 42).fit(Xvalid, yvalid)
eli5.show_weights(perm, feature_names = Xvalid.columns.tolist(), top = 10)

- TotalAmount_Spent is the most important feature according to the above chart, there is a big change in performance metric if we are to random shuffle this feature

In [ ]:
import shap

explainer = shap.Explainer(lin_reg, Xtrain)
shap_values = explainer(Xvalid)

#plotting
shap.summary_plot(shap_values, Xvalid)

- Amount spent of Fish increases with increase in total amount spent
- When customer spend more on Meat, Gold, Sweet and Wines, they tend to spend less on fish products

__2.5 Is there a significant relationship between geographical regional and success of a campaign?__

In [ ]:
df_cmp = df[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']].copy()
cmp_success = df_cmp.sum(axis = 0)
cmp_srate = df_cmp.sum(axis = 0) / len(df_cmp) * 100

sns.barplot(x = df_cmp.columns, y = cmp_success.values)

In [ ]:
plt.pie(x = cmp_srate, labels = df_cmp.columns, autopct = '%1.2f%%', shadow = False, explode = [0, 0.1, 0, 0, 0]);

- AcceptedCmp2 has low success rate as evident from the plots above

In [ ]:
df_cmp = df_orig[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 
                    'Country']].copy()
df_cmp['Total_Accepted'] = df_orig[['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 
                                     'AcceptedCmp5']].sum(axis = 1)
df_cmp.head()

In [ ]:
df_cmp_country = pd.pivot_table(data = df_cmp, index = 'Country', values = 'Total_Accepted', 
                               aggfunc ={'Total_Accepted': ['sum', 'count']})
df_cmp_country['SuccessRate'] = df_cmp_country['sum'] / df_cmp_country['count'] * 100
df_cmp_country.rename(columns = {'count': 'NumCustomers', 'sum': 'NumSuccess'}, inplace = True)
df_cmp_country = df_cmp_country.sort_values(by = 'SuccessRate', ascending = False)
df_cmp_country

- ME-Mexico has the highest overall campaign success rate, but with only 3 customers we can ignore it
- CA-Canada has the next highest success rate

In [ ]:
cm = df_cmp.groupby('Country').agg('mean').sort_values(by = 'Total_Accepted', ascending = False)
cm.style.background_gradient(sns.light_palette('#2ecc71', as_cmap = True))

- We ignore the data from ME
- AcceptedCmp1 has worked well in SP, CA, US
- AcceptedCmp2 doesn't seem to have worked in any of the countries
- AcceptedCmp3 has worked well in almost all the countries
- AcceptedCmp4 has worked well in GER, CA, SP
- AcceptedCmp5 has got good success in AUS, SP, CA

# __Section 03: Data Visualization__

__3.1 Which marketing campaign is most successful?__

- With only 3 data we ignore ME
- Get the mean of campaign success by country

In [ ]:
cm = df_cmp[df_cmp['Country'] != 'ME'].groupby('Country').agg('mean').sort_values(by = 'Total_Accepted', ascending = False)
cm.style.background_gradient(sns.light_palette('#2ecc71', as_cmap = True))

In [ ]:
pd.DataFrame(cm.iloc[:, :5].sum(axis = 0)).style.background_gradient(sns.light_palette('#2ecc71', as_cmap = True))

In [ ]:
pd.DataFrame(cm.iloc[:, :5].sum(axis = 0)).plot(kind = 'bar', rot = 0)
plt.ylabel('Overall Campaign Performance')

- Campaign 'AcceptedCmp3' has performed well in all the countries
- 'Acceptedcmp4' is the next best performing campaign

__3.2 What does the average customer look like for this company?__

- We will look at the average spending of Customer groups by Country

In [ ]:
df_cn = pd.pivot_table(data = df_orig, index = 'Country', values = ['TotalAmount_Spent'], 
                       columns = ['AgeGroup'], aggfunc = ['mean'])
df_cn = df_cn[df_cn.index != 'ME']
df_cn.columns = ['Gen-Z', 'Gen-Y.1', 'Gen-Y.2', 'Gen-X', 'BBoomers']
df_cn.T

In [ ]:
fig, axes = plt.subplots(4, 2, figsize = (20, 20))

for i, (idx, row) in enumerate(df_cn.iterrows()):
    pct = row.values / np.sum(row) * 100
    ax = axes[i // 2, i % 2]
    ax.pie(row, labels = row.index, autopct = '%1.2f%%', radius = 1.2, textprops = {'fontsize': 10}, 
           shadow = True, explode = (pct == max(pct)) * 0.1, startangle = 30)
    ax.set_title(idx)

fig.delaxes(axes[3, 1])
plt.suptitle('Avg. Amount Spent in Countries by Age Demography', fontsize = 15)
plt.show()

In [ ]:
df_cn = pd.pivot_table(data = df_orig, index = 'Country', values = ['TotalAmount_Spent'], 
                       columns = ['Marital_Status'], aggfunc = ['mean'])
df_cn = df_cn[df_cn.index != 'ME']
df_cn.columns = ['Divorced', 'Married', 'Single', 'Together', 'Widow']
df_cn.T

In [ ]:
fig, axes = plt.subplots(4, 2, figsize = (20, 20))

for i, (idx, row) in enumerate(df_cn.iterrows()):
    pct = row.values / np.sum(row) * 100
    ax = axes[i // 2, i % 2]
    ax.pie(row, labels = row.index, autopct = '%1.2f%%', radius = 1.2, textprops = {'fontsize': 10}, 
           shadow = True, explode = (pct == max(pct)) * 0.1, startangle = 30)
    ax.set_title(idx)

fig.delaxes(axes[3, 1])
plt.suptitle('Avg. Amount Spent in Countries by Marital Status', fontsize = 15)
plt.show()

In [ ]:
df_cn = pd.pivot_table(data = df_orig, index = 'Country', values = ['TotalAmount_Spent'], 
                       columns = ['Education'], aggfunc = ['mean'])
df_cn = df_cn[df_cn.index != 'ME']
df_cn.columns = ['Basic', 'Graduation', 'Master', 'PhD', 'Undergraduate']
df_cn.T

In [ ]:
fig, axes = plt.subplots(4, 2, figsize = (20, 20))

for i, (idx, row) in enumerate(df_cn.iterrows()):
    pct = row.values / np.sum(row) * 100
    ax = axes[i // 2, i % 2]
    ax.pie(row, labels = row.index, autopct = '%1.2f%%', radius = 1.2, textprops = {'fontsize': 10}, 
           shadow = True, explode = (pct == max(pct)) * 0.1, startangle = 30)
    ax.set_title(idx)

fig.delaxes(axes[3, 1])
plt.suptitle('Avg. Amount Spent in Countries by Education', fontsize = 15)
plt.show()

In [ ]:
df[['Income', 'Num_Dependants', 'Recency', 'Customer_Age', 'Num_TotalPurchases', 
    'TotalAmount_Spent']].mean().round(1).to_frame(name = 'Average').style.background_gradient(sns.light_palette('#2ecc71', 
                                                                                                                 as_cmap = True))

In [ ]:
df_orig.groupby(['Marital_Status'])['TotalAmount_Spent'].agg('mean').plot.pie(autopct = '%1.2f%%', shadow = True)
plt.title('Avg. Amount Spent by Marital Status');

In [ ]:
df_orig.groupby(['Education'])['TotalAmount_Spent'].agg('mean').plot.pie(autopct = '%1.2f%%', shadow = True)
plt.title('Avg. Amount Spent by Education');

In [ ]:
df_orig.groupby(['AgeGroup'])['TotalAmount_Spent'].agg('mean').plot.pie(autopct = '%1.2f%%', shadow = True)
plt.title('Avg. Amount Spent by Age Group');

__An average customer for this company:__
* is of age 44 
* average amount spent by Gen-Z (under 24 years old) is higher compared to age group 57 - 75
* has atleast 1 dependant (Kid or Teen)
* $52200 is the income he/she earns on an average

* has bought an item in the last 49 days
* spends an average of $605 in purchasing items
* has made about 12 purchases with the company
* Customers with PhD spend the most, while with Master and Graduation spend almost equally
* Customers who are widow seem to be spending more with this company

__3.3 Which products are performing best?__

In [ ]:
mnt_products = [c for c in df.columns if 'Mnt' in c]

def show_value(x):
    a  = np.round(x / 100.0 * np.sum(temp.values), 1)
    return a

temp = df[mnt_products].mean()
plt.pie(temp.values, labels = temp.index, autopct = show_value, shadow = True);

- Customers spent most on Wines
- Next best selling product is the Meat products
- Third best selling product is Gold

__3.4 Which channels are underperforming?__

In [ ]:
cmp_num = [c for c in df.columns if 'AcceptedCmp' in c] + [ c for c in df.columns if ('Num' in c) & 
                                                           (c not in ['Num_Dependants', 'Num_TotalPurchases'])]
ax = df[cmp_num].mean().round(2).to_frame(name = 'Average').sort_values(by = 'Average').plot(kind = 'bar', 
                                                                                            legend = None, rot = 45)
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

- On an average there are more than 5 Store purchases made
- While average number of web visits by customer is 5
- While number of web purchases made is 4 if is an impressive number considering the number of web visits(5)
- There are 2 purchases done via the channels Catalog and Deals respectively
- Campaigns 1-5 have done badly while Campaign 2 is worst of all

In [ ]:
temp = pd.pivot_table(data = df_orig, index = 'AgeGroup', values = ['NumWebVisitsMonth', 'NumWebPurchases'], 
                      aggfunc = 'mean')
temp

In [ ]:
temp = pd.pivot_table(data = df_orig, index = 'AgeGroup', values = ['NumWebVisitsMonth', 'NumWebPurchases'], 
                      aggfunc = 'mean')

fig = plt.figure(figsize = (16, 8))

plt.subplot(2, 2, 1)
pct = temp['NumWebVisitsMonth'].values / np.sum(temp['NumWebVisitsMonth']) * 100
plt.pie(temp['NumWebVisitsMonth'].values, labels = temp.index, autopct = '%1.2f%%', 
        explode = (pct == max(pct)) * 0.1, shadow = True, radius = 1.2)
plt.title('Avg. number of Web visits by Age group', y = 1.1)

plt.subplot(2, 2, 2)
pct = temp['NumWebPurchases'].values / np.sum(temp['NumWebPurchases']) * 100
plt.pie(temp['NumWebPurchases'].values, labels = temp.index, autopct = '%1.2f%%', 
        explode = (pct == max(pct)) * 0.1, shadow = True, radius = 1.2)
plt.title('Avg. number of Web Purchases by Age group', y = 1.1)
plt.show()

In [ ]:
cmp_num = [c for c in df.columns if 'AcceptedCmp' in c]
temp = pd.pivot_table(data = df_orig, index = 'AgeGroup', values = cmp_num, 
                      aggfunc = 'mean')
temp['Total'] = temp.sum(axis = 1)
temp

In [ ]:
def show_value(x):
    a  = np.round(x / 100.0 * np.sum(temp['Total'].values), 2)
    return a
pct = temp['Total'] / np.sum(temp['Total']) * 100

plt.title('Campaign success by Age Group')
temp['Total'].plot.pie(autopct = show_value, explode = (pct == max(pct)) * 0.1, shadow = True);

# Insights and Recommendations

1. Number of total purchases is influenced by the income level of the customers, when there is increase in the number of web/catalog purchases, there is a reduction in store purchases.
2. Customers who spend more on Wines and Meat products tend to spend less on Fish products
    - Also, Married-Graduation category spends more on Fish products than others
3. Overall Campaigns have not done well for this company
    - Although we have ignored the data for Mexico due to limited data, the campaigns have been successful in Mexico, the company can do similar campaigns in other countries to attract customers where it's not successful
    - Campaign2 has not performed well in any of the countries, it's time to revisit how it's been done
    - Campiagn3 is the best performing among all the campaigns
    - Though Campaigns aren't doing well, on an average it's been successful with Gen-Z overall with GenY.1 and Baby Boomers coming 2nd and 3rd respectively - more campaigns should be targeted to drive up the sales among this groups
4. Overall Gen-Z and Baby Boomers are the two groups which have spend more $ in the company
    - This correlates with family with kid/teen spending more
    - Campaigns should be designed to target other age groups
    - Out of 5 web visits on an average 4 purchases were made, more deals should be promoted in web to target customers' web purchases
5. Wines and Meat products are the top 2 best perfoming products in terms of sales
    - Deals and promotions should be carried out to increase the sales of other products

In [ ]:
finish = time()
print(strftime("%H:%M:%S", gmtime(finish - start)))